In [ ]:
"""Download Fashion-MNIST using only the Python standard library + NumPy.

This script downloads the original IDX gzip files, parses them with
`gzip`/`struct` and `numpy.frombuffer`, and saves a single
`data/fashion_mnist.npz` archive with arrays:

  x_train: (60000, 28, 28) uint8
  y_train: (60000,) uint8
  x_test:  (10000, 28, 28) uint8
  y_test:  (10000,) uint8

No TensorFlow / PyTorch required.
"""

from __future__ import annotations

import gzip
import os
import struct
import urllib.request
from pathlib import Path

import numpy as np


BASE_URL = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
FILES = {
    "train_images": "train-images-idx3-ubyte.gz",
    "train_labels": "train-labels-idx1-ubyte.gz",
    "test_images": "t10k-images-idx3-ubyte.gz",
    "test_labels": "t10k-labels-idx1-ubyte.gz",
}


def download(url: str, dest: Path, overwrite: bool = False) -> None:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists() and not overwrite:
        print(f"Skipping download, file exists: {dest}")
        return
    print(f"Downloading {url} -> {dest} ...")
    urllib.request.urlretrieve(url, dest)
    print("Done")


def parse_idx_images(gz_path: Path) -> np.ndarray:
    with gzip.open(gz_path, "rb") as f:
        header = f.read(16)
        magic, num, rows, cols = struct.unpack(
            ">IIII", header
        )  # big-endian unsigned ints
        if magic != 2051:
            raise ValueError(f"Invalid magic number in image file: {magic}")
        buf = f.read(num * rows * cols)
        data = np.frombuffer(buf, dtype=np.uint8)
        data = data.reshape(num, rows, cols)
        return data


def parse_idx_labels(gz_path: Path) -> np.ndarray:
    with gzip.open(gz_path, "rb") as f:
        header = f.read(8)
        magic, num = struct.unpack(
            ">II", header
        )  # big-endian
        if magic != 2049:
            raise ValueError(f"Invalid magic number in label file: {magic}")
        buf = f.read(num)
        labels = np.frombuffer(buf, dtype=np.uint8)
        return labels


def main(dest_dir: str | Path = "data") -> None:
    dest_dir = Path(dest_dir)
    gz_dir = dest_dir / "raw"
    gz_dir.mkdir(parents=True, exist_ok=True)

    # Download files
    for key, fname in FILES.items():
        url = BASE_URL + fname
        out = gz_dir / fname
        download(url, out)

    # Parse
    x_train = parse_idx_images(gz_dir / FILES["train_images"])
    y_train = parse_idx_labels(gz_dir / FILES["train_labels"])
    x_test = parse_idx_images(gz_dir / FILES["test_images"])
    y_test = parse_idx_labels(gz_dir / FILES["test_labels"])

    # Save as compressed npz
    out_npz = dest_dir / "fashion_mnist.npz"
    np.savez_compressed(out_npz, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
    print(f"Wrote {out_npz}:")
    print("  x_train", x_train.shape, x_train.dtype)
    print("  y_train", y_train.shape, y_train.dtype)
    print("  x_test ", x_test.shape, x_test.dtype)
    print("  y_test ", y_test.shape, y_test.dtype)


if __name__ == "__main__":
    import argparse

    p = argparse.ArgumentParser(description="Download Fashion-MNIST (NumPy only)")
    p.add_argument("--dest", default="data", help="Destination directory to save data (default: data)")
    # In Jupyter/IPython the kernel injects extra CLI args (e.g. --f=...), so ignore unknown args
    args, _ = p.parse_known_args()
    main(args.dest)

Skipping download, file exists: data\raw\train-images-idx3-ubyte.gz
Skipping download, file exists: data\raw\train-labels-idx1-ubyte.gz
Skipping download, file exists: data\raw\t10k-images-idx3-ubyte.gz
Skipping download, file exists: data\raw\t10k-labels-idx1-ubyte.gz
Wrote data\fashion_mnist.npz:
  x_train (60000, 28, 28) uint8
  y_train (60000,) uint8
  x_test  (10000, 28, 28) uint8
  y_test  (10000,) uint8
Wrote data\fashion_mnist.npz:
  x_train (60000, 28, 28) uint8
  y_train (60000,) uint8
  x_test  (10000, 28, 28) uint8
  y_test  (10000,) uint8


In [ ]:
# === Import dependencies ===
import numpy as np
import matplotlib.pyplot as plt
from ffnn import FeedForwardNN
from data_loader import load_fashion_mnist

In [ ]:
# === Load Fashion-MNIST dataset ===
X_train, y_train, X_test, y_test = load_fashion_mnist()

# split off 10% of training data for validation
val_size = int(0.1 * X_train.shape[0])
X_val, y_val = X_train[:val_size], y_train[:val_size]
X_train, y_train = X_train[val_size:], y_train[val_size:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
def accuracy(y_true, y_pred):
    # === Calculate accuracy ===
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

In [ ]:
# === Initialize Feedforward Neural Network ===
model = FeedForwardNN(
    layer_sizes=[784, 128, 64, 10],
    activation='relu',
    output_activation='softmax'
)

In [ ]:
# === Set training hyperparameters ===
epochs = 100
lr = 0.1
losses, val_accs = [], []

In [ ]:
# === Training Loop ===
# 1. forward propagation
# 2. computer loss
# 3. backward propagation
# 4. optimize parameteres
# repeat for epochs

for epoch in range(epochs):

    y_pred_train, cache = model.forward(X_train[:5000])
    loss = model.compute_loss(y_train[:5000], y_pred_train)
    
    grads = model.backward(y_train[:5000], cache)
    model.update_params(grads, lr)
    
    train_acc = accuracy(y_train[:5000], y_pred_train)
    y_pred_val, _ = model.forward(X_val[:1000])
    val_acc = accuracy(y_val[:1000], y_pred_val)
    
    losses.append(loss)
    val_accs.append(val_acc)
    
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


In [ ]:
y_pred_test, _ = model.forward(X_test[:2000])
test_acc = accuracy(y_test[:2000], y_pred_test)
print(f"\n✅ Final Test Accuracy: {test_acc:.4f}")

In [ ]:
# === Visualize training loss and validation accuracy ===
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(losses, color='royalblue')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(val_accs, color='green')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy")
plt.grid(True)

plt.tight_layout()
plt.show()